In [1]:
import pandas as pd
import yfinance as yf

ticker = 'BAE.L'
t = yf.Ticker(ticker)

fin = t.quarterly_financials.T
bs = t.quarterly_balance_sheet.T
cf = t.quarterly_cashflow.T

t.financials.T

""


In [2]:
ticker = "HO.PA"
start_year = 2021
end_year = 2025

t = yf.Ticker(ticker)

q_fin = t.quarterly_financials.T
a_fin = t.financials.T
q_bs = t.quarterly_balance_sheet.T
a_bs = t.balance_sheet.T
q_cf = t.quarterly_cashflow.T
a_cf = t.cashflow.T
info = t.info

fin = pd.concat([q_fin, a_fin[~a_fin.index.isin(q_fin.index)]])
bs = pd.concat([q_bs, a_bs[~a_bs.index.isin(q_bs.index)]])
cf = pd.concat([q_cf, a_cf[~a_cf.index.isin(q_cf.index)]])

monthly_dates = pd.date_range(start=f"{start_year}-01-01", end=f"{end_year}-12-31", freq='ME')
fin_dates = fin.index if not fin.empty else []
bs_dates = bs.index if not bs.empty else []
cf_dates = cf.index if not cf.empty else []

q_cf

""


In [3]:
import yfinance as yf
import pandas as pd

def get_fcf_yield(ticker="AIR.PA", start_year=2021, end_year=2025):
    t = yf.Ticker(ticker)

    q_cf = t.quarterly_cashflow.T
    a_cf = t.cashflow.T
    cf = pd.concat([q_cf, a_cf[~a_cf.index.isin(q_cf.index)]])
    cf.sort_index(inplace=True)

    try:
        market_cap = t.info.get("marketCap")
    except:
        market_cap = None

    monthly_dates = pd.date_range(start=f"{start_year}-01-01", end=f"{end_year}-12-31", freq="ME")
    cf_dates = cf.index

    last_fcf = None
    records = []

    for date in monthly_dates:
        cf_date = max([d for d in cf_dates if d <= date], default=None)
        if cf_date and pd.notna(cf.loc[cf_date].get("Free Cash Flow")):
            last_fcf = cf.loc[cf_date].get("Free Cash Flow")

        fcf_yield = last_fcf / market_cap if last_fcf and market_cap else None
        records.append({"date": date, "ticker": ticker, "FCF_Yield": fcf_yield})

    return pd.DataFrame(records)

df = get_fcf_yield()
df.head()

,date,ticker,FCF_Yield
0,2021-01-31,AIR.PA,NaN
1,2021-02-28,AIR.PA,NaN
2,2021-03-31,AIR.PA,NaN
3,2021-04-30,AIR.PA,NaN
4,2021-05-31,AIR.PA,NaN


In [4]:
import yfinance as yf
import pandas as pd

def get_val(s, k):
    if k not in s:
        return None
    val = s[k]
    if isinstance(val, pd.Series):
        return val.iloc[0] if len(val) == 1 else None
    return val if pd.notna(val) else None

def fetch_pe_timeseries(ticker: str, start_year=2022, end_year=2025):
    t = yf.Ticker(ticker)

    fin = pd.concat([t.quarterly_financials.T, t.financials.T]).sort_index()
    bs = pd.concat([t.quarterly_balance_sheet.T, t.balance_sheet.T]).sort_index()

    monthly = pd.date_range(f"{start_year}-01", f"{end_year}-12", freq="M")

    market_cap = t.info.get("marketCap")

    last_net = None
    last_equity = None
    results = []

    for date in monthly:
        f = fin.loc[fin.index[fin.index <= date].max()] if not fin.index[fin.index <= date].empty else pd.Series()
        b = bs.loc[bs.index[bs.index <= date].max()] if not bs.index[bs.index <= date].empty else pd.Series()

        net = get_val(f, "Net Income")
        equity = get_val(b, "Ordinary Shares Number")

        if net is not None:
            last_net = net
        if equity is not None:
            last_equity = equity

        eps = last_net / last_equity if last_net and last_equity else None
        pe = market_cap / (eps * last_equity) if eps and market_cap and last_equity else None

        results.append({
            "date": date,
            "ticker": ticker,
            "P_E": pe
        })

    return pd.DataFrame(results)


df = fetch_pe_timeseries("AIR.PA")
df.head()

C:\Users\ALrig\AppData\Local\Temp\ipykernel_10264\224171485.py:18: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = pd.date_range(f"{start_year}-01", f"{end_year}-12", freq="M")


,date,ticker,P_E
0,2022-01-31,AIR.PA,30.289686
1,2022-02-28,AIR.PA,30.289686
2,2022-03-31,AIR.PA,30.289686
3,2022-04-30,AIR.PA,30.289686
4,2022-05-31,AIR.PA,30.289686


In [5]:
import yfinance as yf 
import pandas as pd

data = yf.download('AIR.PA', start='2024-09-30', end='2025-03-31', interval="1mo", progress=False)
if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.get_level_values(0)
data = data[["Close"]].copy()
data = data.rename(columns={"Close": "price"})
data["return"] = data["price"].pct_change()
data = data.reset_index()
data["ticker"] = 'AIR.PA'
data[["Date", "ticker", "return"]]

Price,Date,ticker,return
0,2024-10-01,AIR.PA,NaN
1,2024-11-01,AIR.PA,0.052797
2,2024-12-01,AIR.PA,0.048929
3,2025-01-01,AIR.PA,0.079726
4,2025-02-01,AIR.PA,-0.008497
5,2025-03-01,AIR.PA,-0.017622


In [5]:

def get_yoy_revenue_growth(ticker="AIR.PA"):
    t = yf.Ticker(ticker)
    df = t.financials.T.copy()
    df = df.sort_index()
    
    df = df[["Total Revenue"]].dropna()
    df = df.reset_index()
    df = df.rename(columns={"index": "date", "Total Revenue": "revenue"})
    df["date"] = pd.to_datetime(df["date"])

    df = df.sort_values("date").reset_index(drop=True)

    # Calcul du YoY Growth
    df["Revenue_Growth_YOY"] = df["revenue"].pct_change(periods=1)
    
    return df


# Exécuter la fonction pour AIR.PA
df_revenue_growth_cagr = get_yoy_revenue_growth("AIR.PA")
df_revenue_growth_cagr

C:\Users\ALrig\AppData\Local\Temp\ipykernel_10264\3143604007.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Revenue_Growth_YOY"] = df["revenue"].pct_change(periods=1)


,date,revenue,Revenue_Growth_YOY
0,2021-12-31,52149000000.0,NaN
1,2022-12-31,58763000000.0,0.126829
2,2023-12-31,65446000000.0,0.113728
3,2024-12-31,69230000000.0,0.057819


In [6]:
ticker = "HO.PA"
t = yf.Ticker(ticker)

# Bilan
print("\n=== t.balance_sheet.T columns ===")
print(list(t.balance_sheet.T.columns))

# Flux de trésorerie
print("\n=== t.cashflow.T columns ===")
print(list(t.cashflow.T.columns))

t.financials.T


=== t.balance_sheet.T columns ===
['Treasury Shares Number', 'Ordinary Shares Number', 'Share Issued', 'Net Debt', 'Total Debt', 'Tangible Book Value', 'Invested Capital', 'Working Capital', 'Net Tangible Assets', 'Capital Lease Obligations', 'Common Stock Equity', 'Total Capitalization', 'Total Equity Gross Minority Interest', 'Minority Interest', 'Stockholders Equity', 'Other Equity Interest', 'Treasury Stock', 'Retained Earnings', 'Additional Paid In Capital', 'Capital Stock', 'Common Stock', 'Total Liabilities Net Minority Interest', 'Total Non Current Liabilities Net Minority Interest', 'Derivative Product Liabilities', 'Non Current Pension And Other Postretirement Benefit Plans', 'Non Current Deferred Taxes Liabilities', 'Long Term Debt And Capital Lease Obligation', 'Long Term Capital Lease Obligation', 'Long Term Debt', 'Current Liabilities', 'Other Current Liabilities', 'Current Debt And Capital Lease Obligation', 'Current Capital Lease Obligation', 'Current Debt', 'Pensionan

,Tax Effect Of Unusual Items,Tax Rate For Calcs,Normalized EBITDA,Total Unusual Items,Total Unusual Items Excluding Goodwill,Net Income From Continuing Operation Net Minority Interest,Reconciled Depreciation,Reconciled Cost Of Revenue,EBITDA,EBIT,...,Operating Income,Operating Expense,Research And Development,Selling General And Administration,Selling And Marketing Expense,General And Administrative Expense,Gross Profit,Cost Of Revenue,Total Revenue,Operating Revenue
2024-12-31,-125172000.0,0.228,3221400000.0,-549000000.0,-549000000.0,1007400000.0,1155400000.0,15202700000.0,2672400000.0,1517000000.0,...,1817000000.0,3556900000.0,1273700000.0,2283200000.0,1590300000.0,692900000.0,5373900000.0,15202700000.0,20576600000.0,20576600000.0
2023-12-31,-124147200.0,0.244,2965100000.0,-508800000.0,-508800000.0,949100000.0,1045100000.0,13662000000.0,2456300000.0,1411200000.0,...,1653700000.0,3112700000.0,1107700000.0,2005000000.0,1384000000.0,621000000.0,4766400000.0,13662000000.0,18428400000.0,18428400000.0
2022-12-31,-36303300.0,0.193,2698300000.0,-188100000.0,-188100000.0,1111500000.0,1058800000.0,13113400000.0,2510200000.0,1451400000.0,...,1442900000.0,3012500000.0,1063600000.0,1948900000.0,1350400000.0,598500000.0,4455400000.0,13113400000.0,17568800000.0,17568800000.0
2021-12-31,-10508400.0,0.139,2383800000.0,-75600000.0,-75600000.0,971200000.0,1062600000.0,12158900000.0,2308200000.0,1245600000.0,...,1235300000.0,2797800000.0,1026900000.0,1770900000.0,1225900000.0,545000000.0,4033100000.0,12158900000.0,16192000000.0,16192000000.0
2020-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
TICKERS = [
    "AIR.PA",    # Airbus
    "SAF.PA",    # Safran
    "HO.PA",     # Thales
    "MTX.DE",    # MTU Aero Engines
    "LDO.MI",    # Leonardo
    "RR.L",      # Rolls-Royce
    "BA.L",     # BAE Systems
    "BA",        # Boeing
    "RTX",       # RTX Corporation
    "LMT",       # Lockheed Martin
    "NOC",       # Northrop Grumman
    "GD",        # General Dynamics
    "TXT",       # Textron
    "HWM",       # Howmet Aerospace
    "SPR",       # Spirit AeroSystems
    "HEI",       # HEICO Corp
    "TDG",       # TransDigm Group
    "BBD-B.TO",  # Bombardier
    "CAE.TO",    # CAE Inc.
    "ERJ",       # Embraer
    "AVAV",      # AeroVironment
]

perf_df = pd.DataFrame({
    "date": ["2022-12-31", "2023-12-31", "2024-12-31"],
    "long_return": [0.382728, 0.147764, 0.281232],
    "short_return": [-0.271525, -0.179943, -0.103217],
    "long_count": [5, 5, 3],
    "short_count": [5, 5, 3],
    "net_return": [0.654252, 0.327708, 0.384448]
})

def compute_benchmark_returns(start: str = "2021-12-31", end: str = None) -> pd.DataFrame:
    price_df = yf.download(TICKERS, start=start, end=end, progress=False)["Close"]
    if isinstance(price_df.columns, pd.MultiIndex):
        price_df = price_df["Close"]
    elif "Close" in price_df.columns:
        price_df = price_df[["Close"]]
        price_df.columns = TICKERS if len(TICKERS) == 1 else [TICKERS[0]]
    returns = price_df.pct_change().dropna()
    benchmark_returns = returns.mean(axis=1).reset_index()
    benchmark_returns.columns = ["date", "benchmark_return"]
    return benchmark_returns

benchmark_df = compute_benchmark_returns()

def adapt_benchmark(benchmark_df, perf_df, start_date):
    benchmark_df = benchmark_df.set_index(pd.to_datetime(benchmark_df['date'])).sort_index()
    dates = pd.to_datetime(perf_df['date'])
    returns = []

    prev = pd.to_datetime(start_date)
    for curr in dates:
        r = benchmark_df.loc[(benchmark_df.index > prev) & (benchmark_df.index <= curr), 'benchmark_return']
        returns.append((1 + r).prod() - 1 if not r.empty else None)
        prev = curr

    return pd.DataFrame({'date': dates, 'benchmark_return': returns})

adapt_benchmark(benchmark_df, perf_df, "2021-12-31")


YF.download() has changed argument auto_adjust default to True


C:\Users\ALrig\AppData\Local\Temp\ipykernel_10264\1918021526.py:41: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = price_df.pct_change().dropna()


,date,benchmark_return
0,2022-12-31,0.144241
1,2023-12-31,0.314599
2,2024-12-31,0.336376
